# Manual Parameter Scan

In [1]:
import itertools
import uproot
import awkward as ak
import numpy as np
import pandas as pd
from coffea import nanoevents
import vector
import fastjet
import math
import matplotlib.pyplot as plt
import matplotlib
import mplhep as hep
import hist
from hist import Hist
from rich.table import Table
from rich.console import Console

In [2]:
matplotlib.rcParams.update({'figure.max_open_warning': 0})
plt.style.use(hep.style.CMS)

In [3]:
vector.register_awkward()

In [4]:
np.set_printoptions(suppress=True)

In [5]:
def inter_isolation(leptons_1, leptons_2, dR=1.6):
    """
    Compute the inter-isolation of each particle. It is supposed to work for one particle per event. The input is:
    - leptons_1: array of leptons for isolation calculation
    - leptons_2: array of all leptons in the events
    - dR: deltaR cut for isolation calculation
    """
    a, b = ak.unzip(ak.cartesian([leptons_1, leptons_2]))
    deltar_mask = a.deltaR(b) < dR
    return (ak.sum(b[deltar_mask].pt, axis=-1) - leptons_1.pt) / leptons_1.pt

def muon_filter(events, nMuons=4):
    """
    Filter events after the TripleMu trigger.
    Cleans muons and electrons.
    Requires at least nMuons with mediumId, pt, dxy, dz, and eta cuts.
    """
    muons = events.Muon
    electrons = events.Electron
    clean_muons = (
        (events.Muon.mediumId)
        & (events.Muon.pt > 3)
        & (abs(events.Muon.dxy) <= 0.02)
        & (abs(events.Muon.dz) <= 0.1)
        & (abs(events.Muon.eta) < 2.4)
    )
    clean_electrons = (
        (events.Electron.mvaFall17V2noIso_WPL)
        & (events.Electron.pt > 3)
        & (
            abs(events.Electron.dxy)
            < 0.05 + 0.05 * (abs(events.Electron.eta) > 1.479)
        )
        & (
            abs(events.Electron.dz)
            < 0.10 + 0.10 * (abs(events.Electron.eta) > 1.479)
        )
        & ((abs(events.Electron.eta) < 1.444) | (abs(events.Electron.eta) > 1.566))
        & (abs(events.Electron.eta) < 2.5)
    )
    muons = muons[clean_muons]
    electrons = electrons[clean_electrons]
    select_by_muons = ak.num(muons, axis=-1) >= nMuons
    events = events[select_by_muons]
    muons = muons[select_by_muons]
    electrons = electrons[select_by_muons]
    return events, electrons, muons

In [6]:
lumi = 59817.406
xsecs = {
    125: 45.2,
    400: 3.16,
    750: 0.4969,
    1000: 0.1845,
}

In [7]:
# Some commone paths
xrd_path = 'root://cmseos.fnal.gov//store/user/chpapage/SUEP_signal_private/'
dataset = lambda mS, decay: f"SUEP-m{mS}-{decay}+RunIIAutumn18-private+MINIAODSIM.root"

In [8]:
decays = ['darkPho', 'darkPhoHad']
scan_points = [125, 400, 750, 1000]

In [14]:
cutflows = {}
pt_leading = []
pt_mean = []
h_i16 = []
for decay in decays:
    for mS in scan_points:
        file = nanoevents.NanoEventsFactory.from_root(
            xrd_path + dataset(mS, decay), 
            schemaclass=nanoevents.NanoAODSchema
        )
        events = file.events()
        name = f'mS-{mS}_{decay}'
        print(f'Working on {name}')
        cutflows[name] = Hist.new.StrCategory(
            [
                '5_3_3_Mass3p8_DZ',  
                '10_5_5_DZ',  
                '12_10_5',  
            ], 
            name='eff', 
            label='eff',
        ).Weight()
        n_total = len(events)
        scale = lumi * xsecs[mS] / n_total

        cutflows[name].fill(
            ['5_3_3_Mass3p8_DZ'], 
            weight=(len(events[events.HLT.TripleMu_5_3_3_Mass3p8_DZ == 1]) / n_total)
        )

        cutflows[name].fill(
            ['10_5_5_DZ'], 
            weight=(len(events[events.HLT.TripleMu_10_5_5_DZ == 1]) / n_total)
        )
        
        cutflows[name].fill(
            ['12_10_5'], 
            weight=(len(events[events.HLT.TripleMu_12_10_5 == 1]) / n_total)
        )
        
#         events, electrons, muons = muon_filter(events)
#         muonsCollection = ak.zip(
#             {
#                 "pt": muons.pt,
#                 "eta": muons.eta,
#                 "phi": muons.phi,
#                 "mass": muons.mass,
#                 "charge": muons.pdgId / (-13),
#             },
#             with_name="Momentum4D",
#         )
#         electronsCollection = ak.zip(
#             {
#                 "pt": electrons.pt,
#                 "eta": electrons.eta,
#                 "phi": electrons.phi,
#                 "mass": electrons.mass,
#                 "charge": electrons.pdgId / (-11),
#             },
#             with_name="Momentum4D",
#         )
#         leptons = ak.concatenate([muonsCollection, electronsCollection], axis=-1)
#         leading_muons = leptons[:, 0]
#         i16 = inter_isolation(leading_muons, leptons)

#         pt10_req = ak.all(muons.pt < 10, axis=-1)
#         Muons_pt10 = muons[pt10_req]
        
#         h = Hist.new.Regular(
#             50, 
#             2, 
#             200, 
#             transform=hist.axis.transform.log, 
#             name=name, 
#             label=name
#         ).Double()
#         h.fill(ak.fill_none(ak.pad_none(muons.pt, 2)[:,0], 0))
#         pt_leading.append(h)
        
#         h = Hist.new.Regular(
#             50, 
#             2, 
#             200, 
#             transform=hist.axis.transform.log, 
#             name=name, 
#             label=name
#         ).Double()
#         h.fill(ak.fill_none(ak.mean(muons.pt, axis=-1), 0))
#         pt_mean.append(h)

#         h = Hist.new.Regular(
#             50, 
#             0, 
#             20,
#             name=name, 
#             label=name
#         ).Double()
#         h.fill(i16)
#         h_i16.append(h)
        
#         num_mu_4 = ak.num(muons) > 4
#         num_mu_6 = ak.num(muons) >= 6
#         num_mu_pt10_4 = ak.num(Muons_pt10) > 4
#         num_mu_pt10_6 = ak.num(Muons_pt10) >= 6
        
#         i64_mu4 = num_mu_4 & (i16 > 1)

#         cutflows[name].fill(ak.sum(num_mu_4) * ['nMu>4'], weight=scale)
#         cutflows[name].fill(ak.sum(num_mu_6) * ['nMu>=6'], weight=scale)
#         cutflows[name].fill(ak.sum(i64_mu4) * ['nMu>4 & I16>1'], weight=scale)
#         cutflows[name].fill(ak.sum(num_mu_pt10_4) * ['nMu>4 & ptMu<10'], weight=scale)
#         cutflows[name].fill(ak.sum(num_mu_pt10_6) * ['nMu>=6 & ptMu<10'], weight=scale)

Working on mS-125_darkPho
Working on mS-400_darkPho
Working on mS-750_darkPho
Working on mS-1000_darkPho
Working on mS-125_darkPhoHad
Working on mS-400_darkPhoHad
Working on mS-750_darkPhoHad
Working on mS-1000_darkPhoHad


### Cutflow tables

In [15]:
table = Table(title="HLT_TripleMu trigger efficiencies")
table.add_column("Sample", justify="center", style="cyan", no_wrap=True, width=22)

year='2018'
samples = []
for decay in decays:
    for mS in scan_points:
        samples += [f'mS-{mS}_{decay}',]

n=3
cols_to_show = [0, 1, 2]# list(range(n,n+3))
for i in cols_to_show:
    col_name = cutflows[samples[0]].axes[0].bin(i)
    style = "green"
    if col_name == "all":
        style = "magenta"
    table.add_column(col_name, justify="center", style=style, no_wrap=True, width=22)

for sample in samples:
    string = []
    estring = []
    for i in range(cutflows[sample].size-1):
        s = f'{cutflows[sample][i].value:0.2f}'
        #es = f'{math.sqrt(cutflows[sample][i].variance):0.2f}'
        if cutflows[sample][i].value < 0.1:
            s = f'{cutflows[sample][i].value:0.3f}'
            #es = f'{math.sqrt(cutflows[sample][i].variance):0.1e}'
        string += [s]
        #estring += [es]
    table.add_row(
        sample,
        #*[f'{string[i]} ± {estring[i]}' for i in cols_to_show], 
        *[f'{string[i]}' for i in cols_to_show], 
    )

console = Console()
console.print(table)

                              HLT_TripleMu trigger efficiencies                              
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃        Sample        ┃   5_3_3_Mass3p8_DZ   ┃      10_5_5_DZ       ┃       12_10_5        ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│    mS-125_darkPho    │         0.19         │        0.076         │        0.031         │
│    mS-400_darkPho    │         0.69         │         0.14         │        0.029         │
│    mS-750_darkPho    │         0.92         │         0.23         │        0.037         │
│   mS-1000_darkPho    │         0.95         │         0.28         │        0.044         │
│  mS-125_darkPhoHad   │        0.025         │        0.020         │        0.009         │
│  mS-400_darkPhoHad   │         0.18         │        0.030         │        0.007         │
│  mS-750_darkPhoHad   │         0.43         │        0.062         │        0.012         │
│  mS-1000_darkPhoHad  │         0.58         │        0.085         │        0.014         │
└──────────────────────┴──────────────────────┴──────────────────────┴──────────────────────┘